<a href="https://colab.research.google.com/github/Cathidumi/fmi_ufpa/blob/main/PrevisaoDia10Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# POR ENQUANTO O CÓDIGO ATUAL SÓ FAZ A PREVISÃO DE UM PARÂMETRO: TEMPERATURA INSTANTÂNEA
import pandas as pd
from fbprophet import Prophet

# Dataframes com cada variável
DFTEM_INS = pd.DataFrame(columns=['INS', 'TEM_INS'])
DFUMD_INS = pd.DataFrame(columns=['INS', 'UMD_INS'])
DFVEN_VEL = pd.DataFrame(columns=['INS', 'VEN_VEL'])

# DataFrame que irá conter a previsão final
DataFrameFinal = pd.DataFrame(columns=['INS', 'TEM_INS', 'UMD_INS', 'VEN_VEL'])

# Lê o .csv com os dados base
dados = pd.read_csv('dados_new4.csv', usecols=['INS', 'HORA', 'TEM_INS', 'UMD_INS', 'VEN_VEL'], parse_dates=['INS'])
dados

,INS,HORA,TEM_INS,VEN_VEL,UMD_INS
0,2012-01-01 00:00:00,0,26.6,1.1,81.0
1,2012-01-01 01:00:00,1,26.3,0.8,81.0
2,2012-01-01 02:00:00,2,26.2,0.7,82.0
3,2012-01-01 03:00:00,3,26.0,0.2,83.0
4,2012-01-01 04:00:00,4,25.6,0.7,85.0
...,...,...,...,...,...
87835,2022-01-07 19:00:00,19,27.2,0.2,83.0
87836,2022-01-07 20:00:00,20,26.8,0.6,86.0
87837,2022-01-07 21:00:00,21,26.7,0.9,87.0
87838,2022-01-07 22:00:00,22,26.1,1.4,90.0


In [48]:
# Gera um .csv por hora do dia dos os dados
for c in range(24):
  teste = dados[dados['HORA'] == c]
  teste.to_csv('HORA/HORA'+str(c)+'.csv', index=False)

In [73]:
# Laço for para fazer a previsão de cada hora
for c in range(24):
  # 1º MOMENTO: TEMPERATURA
  # A cada laço, o algoritmo lê o arquivo correspondente a hora 'c' desejada.
  dadoshorariostemperatura = pd.read_csv('HORA/HORA'+str(c)+'.csv', usecols=['INS', 'TEM_INS'], parse_dates=['INS'])
  dadoshorariostemperatura.rename(columns={'INS': 'ds', 'TEM_INS': 'y'}, inplace = True)
  
  # Faz a previsão usando Prophet
  m = Prophet()
  m.fit(dadoshorariostemperatura)
  future = m.make_future_dataframe(periods=3)
  forecast = m.predict(future)
  forecast.rename(columns={'yhat': 'TEM_INS', 'ds': 'INS'}, inplace = True)

  # Monta um array com os dados da última linha do dataframe com as previsões. colunas 'INS' e 'TEM_INS' 
  arrayteste = [forecast.iloc[-1,0], round(forecast.iloc[-1,18], 1)]
  # Adiciona esse array no DafaFrame final
  DFTEM_INS.loc[c] = arrayteste

  # 2º MOMENTO: UMIDADE
  dadoshorariosumidade = pd.read_csv('HORA/HORA'+str(c)+'.csv', usecols=['INS', 'UMD_INS'], parse_dates=['INS'])
  dadoshorariosumidade.rename(columns={'INS': 'ds', 'UMD_INS': 'y'}, inplace = True)
  
  m = Prophet()
  m.fit(dadoshorariosumidade)
  future = m.make_future_dataframe(periods=3)
  forecast = m.predict(future)
  forecast.rename(columns={'yhat': 'UMD_INS', 'ds': 'INS'}, inplace = True)

  arrayteste = [forecast.iloc[-1,0], round(forecast.iloc[-1,18], 1)]
  DFUMD_INS.loc[c] = arrayteste

  # 3º MOMENTO: VELOCIDADE DO VENTO
  dadoshorariosvento = pd.read_csv('HORA/HORA'+str(c)+'.csv', usecols=['INS', 'VEN_VEL'], parse_dates=['INS'])
  dadoshorariosvento.rename(columns={'INS': 'ds', 'VEN_VEL': 'y'}, inplace = True)
  
  m = Prophet()
  m.fit(dadoshorariosvento)
  future = m.make_future_dataframe(periods=3)
  forecast = m.predict(future)
  forecast.rename(columns={'yhat': 'VEN_VEL', 'ds': 'INS'}, inplace = True)

  arrayteste = [forecast.iloc[-1,0], round(forecast.iloc[-1,18], 1)]
  DFVEN_VEL.loc[c] = arrayteste

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [78]:
# Junção dos dados de cada Dataframe
DataFrameFinal = pd.concat([DFTEM_INS, DFUMD_INS.drop('INS', axis=1), DFVEN_VEL.drop('INS', axis=1)], axis=1)
# Visualização do DataFrame Final
DataFrameFinal

,INS,TEM_INS,UMD_INS,VEN_VEL
0,2022-01-10 00:00:00,25.1,92.4,0.3
1,2022-01-10 01:00:00,25.0,93.6,0.2
2,2022-01-10 02:00:00,24.8,95.0,0.2
3,2022-01-10 03:00:00,24.6,95.8,0.1
4,2022-01-10 04:00:00,24.4,96.5,0.1
5,2022-01-10 05:00:00,24.3,97.0,0.1
6,2022-01-10 06:00:00,24.2,97.4,0.0
7,2022-01-10 07:00:00,24.0,97.6,0.0
8,2022-01-10 08:00:00,24.0,97.6,0.0
9,2022-01-10 09:00:00,23.9,96.6,0.0


In [79]:
# Exporta o DataFrame em um .csv
DataFrameFinal.to_csv('PrevisaoDia20220110Final.csv',  index=False, columns = ['INS', 'TEM_INS', 'UMD_INS', 'VEN_VEL'])